In [15]:
# imports 
import os 
import tensorflow as tf 
import cv2 
from matplotlib import pyplot as plt

# setting GPU to be the device 
gpus = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

## basic functions

In [16]:
# setting up the functions to convert from chars to num and vice versa
vocab = [x for x in "abcdefghijklmnopqrstuvwxyz'?!123456789 "]
charToNum = tf.keras.layers.StringLookup(vocabulary=vocab, oov_token="")
numToChar = tf.keras.layers.StringLookup(vocabulary=charToNum.get_vocabulary(), oov_token="", invert=True)

# facial detection vars 
faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
lastKnownCrop

NameError: name 'lastKnownCrop' is not defined

In [13]:
def faceDetection(img):
    # TROUBLESHOOTING
    # print("max size:",img.shape, img.shape[0] - 3 * padding, img.shape[1] - 3 * padding)
    return faceCascade.detectMultiScale(
        img,
        scaleFactor=1.3,
        minNeighbors=5,
        minSize=(30, 30),
    )
def cropForFace(img):
    global lastKnownCrop
    rects = faceDetection(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY))
    # TO DO 
    # ADD ERROR HANDLING FOR IF THERE IS MULTIPLE FACES
    for (x,y,w,l) in rects: 
        # TROUBLESHOOTING
        # print(img.shape, x,y,w,l)
        lastKnownCrop = (x,y,w,l)
    y1 = lastKnownCrop[1] 
    x1 = lastKnownCrop[0]
    y2 = y1 + lastKnownCrop[3] 
    x2 = x1 + lastKnownCrop[2]
    return img[y1:y2, x1:x2]